In [1]:
%load_ext wstl

In [11]:
%%markdown

## Load Data from HL7v2Store
The following line magic command allows you to load data directly from a HL7v2Store and store the results either in a python variable or persist them to disk.


## Load Data from HL7v2Store
The following line magic command allows you to load data directly from a HL7v2Store and store the results either in a python variable or persist them to disk.


In [2]:
%load_hl7v2_datastore?

Docstring:
::

  %load_hl7v2_datastore --project_id PROJECT_ID --region REGION
                            --dataset_id DATASET_ID --hl7v2_store_id
                            HL7V2_STORE_ID [--api_version {v1,v1beta1}]
                            [--filter FILTER]
                            [--dest_file_name DEST_FILE_NAME]

Load parsed HL7v2 massage from the HL7v2 Store specified.

optional arguments:
  --project_id PROJECT_ID
                        ID of the GCP project that the HL7v2 Store belongs to.
  --region REGION       Region of the HL7v2 Store.
  --dataset_id DATASET_ID
                        ID of the dataset that the HL7v2 store belongs to.
  --hl7v2_store_id HL7V2_STORE_ID
                        ID of the HL7v2 store to load data from.
  --api_version <{v1,v1beta1}>
                        The version of healthcare api to call. Default to v1.
  --filter FILTER       filter: string, Restricts messages returned to those
                        matching a filter. Syntax:

In [2]:
test = %load_hl7v2_datastore --project_id bebinu-dev-test --region us-central1 --dataset_id notebook --hl7v2_store_id notebook

In [12]:
%%markdown

## Load JSON data directly from Google Cloud Storage

The following line magic command loads JSON data from a GCS bucket and 
allows the user to either save the results to a python variable, persist them to disk, or print the results.


## Load JSON data directly from Google Cloud Storage

The following line magic command loads JSON data from a GCS bucket and 
allows the user to either save the results to a python variable, persist them to disk, or print the results.


In [4]:
%load_hl7v2_gcs?

Docstring:
::

  %load_hl7v2_gcs --bucket_name BUCKET_NAME --source_blob_name
                      SOURCE_BLOB_NAME [--dest_file_name DEST_FILE_NAME]

Load and return parsed HL7v2 massage from the blob in a GCS bucket specified.

optional arguments:
  --bucket_name BUCKET_NAME
                        The name of the GCS bucket to load data from.
  --source_blob_name SOURCE_BLOB_NAME
                        The name of the blob to load.
  --dest_file_name DEST_FILE_NAME
                        The destination file path to store the loaded data. If
                        not provided, the result will be directly returned to
                        the IPython kernel.
File:      /opt/conda/lib/python3.7/site-packages/wstl/magics/wstl.py


In [5]:
%load_hl7v2_gcs --bucket_name bebinu-dev-test-data --source_blob_name hl7v2/simhos/ndjson/adt_a01_msh.hl7.fhir.input.json

<IPython.core.display.JSON object>

In [23]:
%%markdown

## Evaluate whistle mapping language
* allows the user to execute whistle mapping language scripts and to immediately see the result of executing the script
* result can be displayed in output or stored into a python variable.
* inputs and library configs can be read from local filesystem or Google Cloud Storage
* outputs a FHIR transaction bundle


## Evaluate whistle mapping language
* allows the user to execute whistle mapping language scripts and to immediately see the result of executing the script
* result can be displayed in output or stored into a python variable.
* inputs and library configs can be read from local filesystem or Google Cloud Storage
* outputs a FHIR transaction bundle


In [8]:
%%wstl?

Docstring:
::

  %wstl [--input INPUT] [--library_config LIBRARY_CONFIG]
            [--output OUTPUT]

Cell magic to evaluate whistle mapping language from iPython kernel.

optional arguments:
  --input INPUT         The input. Supports the following prefix notations:
                        py://<name_of_python_variable>
                        json://<inline_json_object_or_array> : python inline
                        dict and list expressions are supported. e.g.
                        json://{"field":"value"} or
                        json://[{"first":"value"},{"second":"value"}]
                        file://<path_to_local_file_system> , supports glob
                        wildcard expressions and will only load .json or
                        .ndjson file extensions. Each json object/list defined
                        within an ndjson will be a separate input to the
                        mapping.
  --library_config LIBRARY_CONFIG
                        Path to the dir

In [2]:
%%wstl --output ccf --input gs://bebinu-dev-test-data/hl7v2/simhos/ndjson/adt_a01_msh.hl7.fhir.input.json --library_config gs://bebinu-dev-test-data/hl7v2/library_functions/*.wstl

$this : Process_ADT_A01($root.ADT_A01);

// --input gs://bebinu-dev-test-data/hl7v2/simhos/ndjson/adt_a01_msh.hl7.fhir.input.json
// --input file://adt_a01_msh.hl7.fhir.input.json
// --input py://b
    
// Reads from Google Storage
// %%wstl --output resources --input gs://bebinu-dev-test-daa/hl7v2/simhos/ndjson/adt_a01_msh.hl7.fhir.input.json --library_config gs://bebinu-dev-test-data/hl7v2/library_functions/*.wstl
// gs://bebinu-dev-test-data/hl7v2/library_functions/*

// Reads from local file system
// %%wstl --output resources --input file://examples/adt_a01_msh.hl7.fhir.input.json --library_config file:///google/src/cloud/bebinu/magic_commands/google3/third_party/cloud_healthcare_data_harmonization/mapping_configs/hl7v2_fhir_stu3/projector_library/*

// Description: Define HL7v2 (version 2.9) to FHIR STU3 transformation
//
// Argument(s):
//   ADT_A01 HL7v2 Message Type (version 2.9)
//   ADT_A01 Segments covered in this mapping configuration are: MSH
//
// Output(s):
//   FHIR STU3 Resources : http://www.hl7.org/fhir/stu3/resourcelist.html
// List of Generated FHIR Resources:
//   Bundle: https://www.hl7.org/fhir/stu3/bundle.html

def Process_ADT_A01(ADT_A01) {
  var MH : MSH_MessageHeader(ADT_A01.MSH);
  out Resources : MH;
  out Resources : HD_Provenance(ADT_A01.MSH.3, MH);
  out Resources : MSH_Bundle(ADT_A01.MSH.10);
}

// post Output_Transaction


<IPython.core.display.JSON object>

In [24]:
%%markdown

## Validate FHIR Resources 
* user can create a FHIR resource using whistle and then run the fhir_validate magic command to validate that the resource. 
* the fhir_validate line magic will perform structure, cardinality, and value domains validation on the resource. 



## Validate FHIR Resources 
* user can create a FHIR resource using whistle and then run the fhir_validate magic command to validate that the resource. 
* the fhir_validate line magic will perform structure, cardinality, and value domains validation on the resource. 


In [3]:
%%wstl --output validation_resource --input gs://bebinu-dev-test-data/hl7v2/simhos/ndjson/adt_a01_msh.hl7.fhir.input.json --library_config gs://bebinu-dev-test-data/hl7v2/library_functions/*.wstl

$this : Process_ADT_A01($root.ADT_A01);

// Description: Define HL7v2 (version 2.9) to FHIR STU3 transformation
//
// Argument(s):
//   ADT_A01 HL7v2 Message Type (version 2.9)
//   ADT_A01 Segments covered in this mapping configuration are: MSH
//
// Output(s):
//   FHIR STU3 Resources : http://www.hl7.org/fhir/stu3/resourcelist.html

def Process_ADT_A01(ADT_A01) {
  var MH : MSH_MessageHeader_local(ADT_A01.MSH);
  $this : MH;
}

def MSH_MessageHeader_local (MSH) {
  id : $UUID();
  resourceType : "MessageHeader";
  
  source.name : MSH.3.1;
    
  // uncomment the line below to produce a validation error
  // source.software : 1;
    
}


<IPython.core.display.JSON object>

In [4]:
%fhir_validate --version stu3 --input py://validation_resource

<IPython.core.display.JSON object>